# Actividad 1 – Big Data  
## Creación de una Base de Datos Analítica para la empresa Clínica del Play S.A.S.  
### Estudiante: Justin Beckham Cardona Yepes  
### Materia: Big Data  
### Institución: IU Digital de Antioquia  

# 1. Definición del Problema

La empresa **Clínica del Play S.A.S.** se dedica al análisis, diagnóstico y optimización de ventas de videojuegos exclusivos de consolas PlayStation.  
Actualmente trabaja con tiendas de videojuegos, distribuidores minoristas y desarrolladores independientes.

La empresa enfrenta un problema:  
No posee una **base de datos analítica unificada** que permita realizar consultas estratégicas sobre:

- Juegos más vendidos.  
- Comparación de ventas entre PS3, PS4 y PS5.  
- Tendencias por género.  
- Desempeño por editora (Publisher).  
- Variación de ventas entre regiones del mundo.

Para resolver esta necesidad, se desarrollará una base de datos analítica basada en un dataset real que permitirá tomar decisiones basadas en datos.

Este proyecto construirá:

✅ Un modelo entidad–relación  
✅ Base de datos analítica  
✅ Carga del dataset  
✅ Consultas SQL  
✅ Documentación completa en Jupyter Notebook  


# 2. Dataset Seleccionado

Se utiliza el siguiente dataset de Kaggle:

**PlayStation Sales and Metadata (PS3, PS4, PS5)**  
Enlace: https://www.kaggle.com/datasets/gvidalguiresse/playstation-sales-and-metadata-ps3ps4ps5/data

## Motivo de selección

Este dataset contiene información esencial para la operación de Clínica del Play S.A.S., como:

- Nombre del videojuego  
- Plataforma (PS3, PS4, PS5)  
- Género  
- Editora  
- Año de lanzamiento  
- Ventas por región  
- Ventas globales  

## Variables relevantes

| Variable | Descripción | Utilidad |
|---------|-------------|----------|
| Name | Nombre del juego | Identificación |
| Platform | PS3/PS4/PS5 | Comparación entre generaciones |
| Genre | Género del juego | Identificar tendencias |
| Publisher | Empresa editora | Evaluar desempeño por marca |
| Year_of_Release | Año de lanzamiento | Análisis temporal |
| Global_Sales | Ventas totales | Indicador principal |
| NA_Sales, EU_Sales, JP_Sales | Ventas por región | Segmentación geográfica |

Estas variables permiten análisis completos para recomendaciones comerciales y de mercado.


## 🧩 Modelo Entidad-Relación

El modelo entidad-relación propuesto para la empresa **Clínica del Play S.A.S.** tiene como objetivo estructurar una base de datos analítica que permita gestionar y analizar la información de las ventas de videojuegos de manera eficiente.  
El modelo está compuesto por **tres entidades principales**: **Desarrolladora**, **Juego** y **Venta**, las cuales se relacionan entre sí para reflejar la realidad del negocio.

---

### **1. Entidad: Desarrolladora**
Representa a las empresas encargadas de crear y publicar los videojuegos.

**Atributos:**
- `id_desarrolladora` (PK): Identificador único de la desarrolladora.  
- `nombre`: Nombre de la empresa desarrolladora o publisher.  
- `pais`: País de origen (puede ser nulo o estimado).  
- `anio_fundacion`: Año en el que fue fundada la empresa.

---

### **2. Entidad: Juego**
Contiene la información específica de cada videojuego disponible en el mercado.

**Atributos:**
- `id_juego` (PK): Identificador único del videojuego.  
- `titulo`: Nombre del videojuego.  
- `genero`: Categoría o tipo del juego (acción, deportes, aventura, etc.).  
- `plataforma`: Consola o sistema en el que se lanzó (PS4, PS5, Xbox, etc.).  
- `fecha_lanzamiento`: Fecha aproximada de lanzamiento.  
- `id_desarrolladora` (FK): Relación con la entidad **Desarrolladora**, indicando qué empresa lo desarrolló.

**Relación:**  
Una **Desarrolladora puede crear varios Juegos**, pero **cada Juego pertenece a una sola Desarrolladora**.  
➡️ **Cardinalidad:** 1 a N (Desarrolladora → Juego)

---

### **3. Entidad: Venta**
Registra los datos de ventas de cada videojuego por región o a nivel global.

**Atributos:**
- `id_venta` (PK): Identificador único de la venta.  
- `id_juego` (FK): Referencia al videojuego vendido.  
- `region`: Región donde se realizó la venta (ej. NA, EU, JP, Other o Global).  
- `anio`: Año en que se registró la venta.  
- `unidades_vendidas`: Número de unidades vendidas (en millones).  
- `ingresos`: Total de ingresos generados (en millones de dólares o equivalente).

**Relación:**  
Un **Juego puede tener múltiples registros de Venta** (por distintas regiones o años), pero **cada Venta corresponde a un solo Juego**.  
➡️ **Cardinalidad:** 1 a N (Juego → Venta)

---

### **📈 Relaciones Finales del Modelo**
- **Desarrolladora (1) → (N) Juego**  
- **Juego (1) → (N) Venta**

---

### **💡 Beneficios del Modelo**
- Permite realizar análisis detallados de ventas por **juego, región, género o plataforma**.  
- Facilita identificar las **desarrolladoras con mejor rendimiento comercial**.  
- Proporciona una estructura flexible para **consultas analíticas** y **dashboards** en herramientas como Power BI o SQL.

### Diagrama ERD – Clínica del Play

A continuación se muestra el modelo entidad–relación diseñado para la base de datos analítica de la Clínica del Play:

![ERD Clínica del Play](./MODELOER.png)



## 4. Creación y carga de la base de datos en Databricks

En esta sección se implementa la base de datos analítica de la **Clínica del Play S.A.S.**,  
siguiendo el modelo entidad–relación previamente diseñado.

Se utiliza **Apache Spark SQL** dentro de **Databricks Community Edition** para:

1. Crear la base de datos `clinica_play_db`.
2. Crear las tres tablas: `desarrolladora`, `juego` y `venta`.
3. Cargar los datos del dataset `playdataset.csv` (descargado desde Kaggle).
4. Mostrar las primeras filas de cada tabla para validar la carga.

El objetivo es disponer de una base de datos analítica que permita realizar consultas
posteriores sobre las ventas de videojuegos, las desarrolladoras y sus plataformas.

# Crear base de datos y instalación PYSPARK



In [23]:
%pip install pyspark
%pip install tabulate

from pyspark.sql import SparkSession

# Crear la sesión de Spark
spark = SparkSession.builder.appName("BigData2025").getOrCreate()

# Leer el archivo con el delimitador correcto
ruta = "playdataset.csv"

df = spark.read.option("header", True).option("inferSchema", True).option("delimiter", ";").csv(ruta)

# Mostrar las primeras filas para verificar
df.show(5, truncate=False)

spark.sparkContext.setLogLevel("ERROR")



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
+------------------------------+-------+--------------------------+--------------+--------------+-------------+-----------+---------+---------+-----------+-----------+------------+-----------+------+-------------+----------+---------------+------------------------------------------------------------------------------------+----+----+
|Game                          |Console|Name                      |Publisher     |Developer     |Total Shipped|Total Sales|NA Sales |PAL Sales|Japan Sales|Other Sales|Release Date|Last Update|rating|ratings_count|metacritic|genres         |platforms                                         

## 🔹 Punto 3: Creación de la base de datos y carga de datos

En esta sección se crea la base de datos analítica denominada **BigData2025** dentro del entorno de Spark.  
Posteriormente, se cargan los datos del dataset *PlayStation Sales and Metadata (PS3, PS4, PS5)* y se visualizan los primeros registros como evidencia.  

El objetivo de este paso es demostrar la correcta creación del esquema, la estructura de la tabla y la carga inicial de datos desde un archivo CSV, utilizando **PySpark**.



In [2]:
# ============================================
# 1. CREACIÓN DE LA BASE DE DATOS (SCHEMA)
# ============================================

# Crear la base de datos si no existe
spark.sql("CREATE DATABASE IF NOT EXISTS bigdata2025")

# Seleccionar la base de datos para usarla
spark.sql("USE bigdata2025")

print("✅ Base de datos 'bigdata2025' creada y seleccionada correctamente.")



✅ Base de datos 'bigdata2025' creada y seleccionada correctamente.


Se crea el esquema bigdata2025 donde residirán todas las tablas del proyecto.
Esta base representa el entorno de datos de la empresa Clínica del Play, que gestiona información de ventas de videojuegos y desarrolladoras

In [17]:
# ============================================
# 2. CARGA DE DATOS DESDE CSV
# ============================================


from pyspark.sql import SparkSession
from tabulate import tabulate

# Crear la sesión de Spark (si no está creada)
spark = SparkSession.builder.appName("BigData2025").getOrCreate()

# Ruta del archivo CSV
ruta = "playdataset.csv"

# Cargar el CSV con las opciones adecuadas
df = (
    spark.read
    .option("header", "true")          # Usa la primera fila como encabezado
    .option("inferSchema", "true")     # Detecta automáticamente los tipos de datos
    .option("sep", ";")                # Define el separador correcto
    .csv(ruta)
)

# Eliminar columnas vacías (_c18, _c19, etc.)
columnas_validas = [c for c in df.columns if not c.startswith("_c")]
df = df.select(columnas_validas)

# Crear la vista temporal para futuras consultas
df.createOrReplaceTempView("playdataset")

# Mostrar las primeras filas en formato tabular
print(tabulate(df.limit(5).collect(), headers=df.columns, tablefmt="grid"))


+--------------------------------+-----------+----------------------------+----------------+----------------+-----------------+---------------+------------+-------------+---------------+---------------+----------------+---------------+----------+-----------------+--------------+-----------------+--------------------------------------------------------------------------------------+
| Game                           | Console   | Name                       | Publisher      | Developer      |   Total Shipped |   Total Sales |   NA Sales |   PAL Sales |   Japan Sales |   Other Sales | Release Date   | Last Update   |   rating |   ratings_count |   metacritic | genres          | platforms                                                                            |
+================================+===========+============================+================+================+=================+===============+============+=============+===============+===============+================+===========

Se carga correctamente el archivo CSV que contiene información de los títulos de PlayStation, sus géneros, plataformas, regiones de venta y puntuaciones.
Se utiliza inferSchema=True para que Spark detecte automáticamente los tipos de datos de cada columna.

In [7]:
# ============================================
# 3. CREACIÓN DE TABLA DENTRO DE LA BASE DE DATOS
# ============================================

# Guardar la tabla en el esquema bigdata2025
#df.write.mode("overwrite").saveAsTable("bigdata2025.playdataset")

#print("✅ Tabla 'playdataset' creada correctamente dentro de la base de datos 'bigdata2025'.")

# Crear una vista temporal
df.createOrReplaceTempView("playdataset")

print("✅ Vista temporal 'playdataset' creada correctamente dentro del entorno local.")



✅ Vista temporal 'playdataset' creada correctamente dentro del entorno local.


La tabla playdataset fue creada dentro del esquema bigdata2025.
Esta tabla permitirá realizar consultas SQL directamente sobre los datos cargados del archivo CSV.

In [18]:
# ============================================
# 4. VERIFICACIÓN DE LA CARGA DE DATOS
# ============================================

# Mostrar las primeras filas directamente con SQL
spark.sql("SELECT * FROM playdataset LIMIT 5").toPandas()



,Game,Console,Name,Publisher,Developer,Total Shipped,Total Sales,NA Sales,PAL Sales,Japan Sales,Other Sales,Release Date,Last Update,rating,ratings_count,metacritic,genres,platforms
0,PS3_Grand Theft Auto V,PS3,Grand Theft Auto V,Rockstar Games,Rockstar North,0.0,20320000.0,6370000.0,9850000.0,990000.0,3120000.0,17/09/2013,None,4.47,7225.0,92.0,Action,"PC, PlayStation 5, Xbox One, PlayStation 4, Xb..."
1,PS4_Grand Theft Auto V,PS4,Grand Theft Auto V,Rockstar Games,Rockstar North,0.0,19390000.0,6060000.0,9710000.0,600000.0,3020000.0,18/11/2014,3/01/2018,4.47,7225.0,92.0,Action,"PC, PlayStation 5, Xbox One, PlayStation 4, Xb..."
2,PS4_Call of Duty: Black Ops 3,PS4,Call of Duty: Black Ops 3,Activision,Treyarch,0.0,15090000.0,6180000.0,6050000.0,410000.0,2440000.0,6/11/2015,14/01/2018,None,None,NaN,None,None
3,PS4_Red Dead Redemption 2,PS4,Red Dead Redemption 2,Rockstar Games,Rockstar Games,0.0,13940000.0,5260000.0,6210000.0,210000.0,2260000.0,26/10/2018,2/11/2018,4.59,5357.0,96.0,Action,"PC, Xbox One, PlayStation 4"
4,PS3_Call of Duty: Black Ops II,PS3,Call of Duty: Black Ops II,Activision,Treyarch,0.0,13800000.0,4990000.0,5880000.0,650000.0,2280000.0,13/11/2012,None,3.96,1417.0,74.0,"Shooter, Action","PC, Xbox One, Xbox 360, PlayStation 3, Wii U"


Se muestran los primeros cinco registros de la tabla playdataset, confirmando la correcta carga de los datos en la base de datos.
Esto evidencia que la estructura de la base y los valores están listos para su análisis en el punto siguiente.

## PUNTO 4 — Evidencia con Consultas SQL
📝 Objetivo

Evidenciar mediante consultas SQL la correcta creación, comprensión y manipulación de la base de datos usando PySpark.
Se realizarán tres consultas:
1️⃣ Conteo de registros
2️⃣ Nombres y tipos de columnas
3️⃣ Consulta con filtro.

In [22]:
from pyspark.sql import SparkSession

# Crear la sesión de Spark
spark = SparkSession.builder.appName("Actividad_BigData_Punto4").getOrCreate()

# Cargar el dataset (delimitado por ;)
df = spark.read.csv("playdataset.csv", header=True, sep=";", inferSchema=True)

# Registrar la tabla temporal para consultas SQL
df.createOrReplaceTempView("playdataset")

# Mostrar una vista previa de los datos
df.show(5, truncate=False)


+------------------------------+-------+--------------------------+--------------+--------------+-------------+-----------+---------+---------+-----------+-----------+------------+-----------+------+-------------+----------+---------------+------------------------------------------------------------------------------------+----+----+
|Game                          |Console|Name                      |Publisher     |Developer     |Total Shipped|Total Sales|NA Sales |PAL Sales|Japan Sales|Other Sales|Release Date|Last Update|rating|ratings_count|metacritic|genres         |platforms                                                                           |_c18|_c19|
+------------------------------+-------+--------------------------+--------------+--------------+-------------+-----------+---------+---------+-----------+-----------+------------+-----------+------+-------------+----------+---------------+------------------------------------------------------------------------------------+---

El dataset fue cargado correctamente y convertido en una vista temporal llamada playdataset, sobre la cual ejecutaremos las consultas SQL.
Esto permite tratar los datos como una tabla y realizar análisis mediante lenguaje SQL.

## Consulta 1: Conteo de registros

In [12]:
# Importamos las librerías necesarias
import pandas as pd

# 🟢 1️⃣ Conteo de registros
conteo = spark.sql("SELECT COUNT(*) AS total_registros FROM playdataset")
display(pd.DataFrame(conteo.collect(), columns=conteo.columns))


,total_registros
0,4963



**🧠 Interpretación:**  
El resultado muestra el número total de registros (filas) que contiene la tabla `playdataset`.  
Esto permite verificar que el conjunto de datos fue cargado correctamente y no tiene pérdidas durante la importación.


In [13]:
# 🟢 2️⃣ Descripción de columnas y tipos de datos
descripcion = spark.sql("DESCRIBE TABLE playdataset")
display(pd.DataFrame(descripcion.collect(), columns=descripcion.columns))

,col_name,data_type,comment
0,Game,string,None
1,Console,string,None
2,Name,string,None
3,Publisher,string,None
4,Developer,string,None
5,Total Shipped,string,None
6,Total Sales,string,None
7,NA Sales,string,None
8,PAL Sales,double,None
9,Japan Sales,double,None


**🧠 Interpretación:**  
Esta consulta devuelve los nombres de las columnas, sus tipos de datos y si permiten valores nulos.  
De esta manera se evidencia que la estructura de la tabla fue interpretada correctamente por Spark, y que cada campo,está asociado con su tipo de dato adecuado (texto, número, fecha, etc.).

In [21]:
# Consulta filtrada: registros donde el Publisher es 'Rockstar Games'
spark.sql("""
SELECT *
FROM playdataset
WHERE Publisher = 'Rockstar Games'
LIMIT 5
""").toPandas().style.hide(axis='index')

Game,Console,Name,Publisher,Developer,Total Shipped,Total Sales,NA Sales,PAL Sales,Japan Sales,Other Sales,Release Date,Last Update,rating,ratings_count,metacritic,genres,platforms,_c18,_c19
PS3_Grand Theft Auto V,PS3,Grand Theft Auto V,Rockstar Games,Rockstar North,0.0,20320000.0,6370000.0,9850000.000000,990000.000000,3120000.000000,17/09/2013,None,4.47,7225.0,92.000000,Action,"PC, PlayStation 5, Xbox One, PlayStation 4, Xbox Series S/X, Xbox 360, PlayStation 3",None,None
PS4_Grand Theft Auto V,PS4,Grand Theft Auto V,Rockstar Games,Rockstar North,0.0,19390000.0,6060000.0,9710000.000000,600000.000000,3020000.000000,18/11/2014,3/01/2018,4.47,7225.0,92.000000,Action,"PC, PlayStation 5, Xbox One, PlayStation 4, Xbox Series S/X, Xbox 360, PlayStation 3",None,None
PS4_Red Dead Redemption 2,PS4,Red Dead Redemption 2,Rockstar Games,Rockstar Games,0.0,13940000.0,5260000.0,6210000.000000,210000.000000,2260000.000000,26/10/2018,2/11/2018,4.59,5357.0,96.000000,Action,"PC, Xbox One, PlayStation 4",None,None
PS3_Grand Theft Auto IV,PS3,Grand Theft Auto IV,Rockstar Games,Rockstar North,0.0,10570000.0,4790000.0,3730000.000000,440000.000000,1620000.000000,29/04/2008,None,4.26,3068.0,95.000000,Action,"PC, Xbox One, Xbox 360, PlayStation 3",None,None
PS3_Red Dead Redemption,PS3,Red Dead Redemption,Rockstar Games,Rockstar San Diego,0.0,6570000.0,2820000.0,2580000.000000,170000.000000,1000000.000000,11/10/2011,None,4.42,2158.0,95.000000,"Shooter, Action","PC, Xbox One, PlayStation 4, Nintendo Switch, Xbox 360, PlayStation 3",None,None


Interpretación:
Aquí se muestran los primeros 5 registros donde el Publisher es "Rockstar Games".
Esta consulta permite validar que los filtros funcionan correctamente y que la información contenida en el dataset se puede explorar con precisión a través de SQL en Spark.